# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [21]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  a_div = -3 *a**2 - 0.5 * a**-0.5
  b_div = 3 * cos(3*b) + 2.5 * b**1.5
  c_div = c ** -2
  return [a_div, b_div, c_div] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [20]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
def approx_gradf(a, b, c):
    h = 0.000000001
    a_div = (f(a+h, b, c) - f(a, b, c)) / h
    b_div = (f(a, b+h, c) - f(a, b, c)) / h
    c_div = (f(a, b, c+h) - f(a, b, c)) / h
    return [a_div, b_div, c_div]

numerical_grad = approx_gradf(2, 3, 4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554623700802
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250022721587811


In [26]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
def symmeterical_approx_gradf(a, b, c):
    h = 0.000000001
    a_div = (f(a+h, b, c) - f(a-h, b, c)) / (2*h)
    b_div = (f(a, b+h, c) - f(a, b-h, c)) / (2*h)
    c_div = (f(a, b, c+h) - f(a, b, c-h)) / (2*h)
    return [a_div, b_div, c_div]

numerical_grad2 = symmeterical_approx_gradf(2, 3, 4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554623700802
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250022721587811


## section 2: support for softmax

In [23]:
import torch
import random
import math

class Value:
    def __init__(self, data, __children=(), op="", label=""):
        self.data = data
        self.grad = 0.0
        self.backward = lambda: None
        self.prev = set(__children)
        self.op = op
        self.label = label

    def __repr__(self):
        return f"Value=(data={self.data})"

    def __add__(self, other):
        if type(other) != Value:
            raise ValueError(f"Other is not of type 'Value', other is of type '{type(other)}'")

        out = Value(self.data + other.data, (self, other), '+', label="v")

        def backward():
            self.grad += out.grad
            other.grad += out.grad
        
        out.backward = backward
        return out
    
    def __sub__(self, other):
        if type(other) != Value:
            raise ValueError(f"Other is not of type 'Value', other is of type '{type(other)}'")
    
        out = Value(self.data - other.data, (self, other), '-', label="v")

        def backward():
            self.grad += out.grad
            other.grad += -out.grad
        
        out.backward = backward
        return out
    
    def __mul__(self, other):
        if type(other) != Value:
            raise ValueError(f"Other is not of type 'Value', other is of type '{type(other)}'")

        out = Value(self.data * other.data, (self, other), '*', label="v")

        def backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out.backward = backward
        return out

    def __neg__(self):
        return self.__mul__(Value(-1))
    
    def __truediv__(self, other):
        if type(other) != Value:
            raise ValueError(f"Other is not of type 'Value', other is of type '{type(other)}'")

        out = Value(self.data / other.data, (self, other), "/", label="v")

        def backward():
            self.grad += other.data**-1 * out.grad
            other.grad += (-1 * (self.data * other.data**-2)) * out.grad

        out.backward = backward
        return out
    
    def __pow__(self, other):
        if type(other) != Value:
            raise ValueError(f"Other is not of type 'Value', other is of type '{type(other)}'")

        out = Value(self.data ** other.data, (self, other), "**", label="v")

        def backward():
            self.grad += (other.data * self.data ** (other.data-1)) * out.grad
        
        out.backward = backward
        return out

    def exp(self):
        out = Value(math.e ** self.data, (self,), "exp", label="v")

        def backward():
            self.grad += out.data * out.grad

        out.backward = backward
        return out
    
    def log(self):
        out = Value(math.log(self.data), (self,), "log", label="v")

        def backward():
            self.grad += (1/self.data) * out.grad
        
        out.backward = backward
        return out

    def tanh(self):
        tanh_res = (math.exp(2*self.data) - 1) / (math.exp(2*self.data) + 1)
        out = Value(tanh_res, (self,), "tanh", label="o")

        def backward():
            self.grad += (1 - t**2) * out.grad
        
        out.backward = backward
        return out
    
    def backwards(self):
        self.grad = 1.0
        topo = []
        visited = set()
        def build_topo(node):
            if node in visited:
                return

            visited.add(node)
            for child in node.prev:
                build_topo(child)

            topo.append(node)
        
        build_topo(self)
        for node in reversed(topo):
            node.backward()

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1), label="w") for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1), label="b")

    def __call__(self, x):
        out = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return out.tanh()
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        if len(outs) == 1:
            return outs[0]

        return outs
    
    def parameters(self):
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        
        return params

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        
        return x
    
    def parameters(self):
        params = []
        for layer in self.layers:
            ps = layer.parameters()
            params.extend(ps)
        
        return params

In [49]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts, Value(0))
  out = [count / denominator for count in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backwards()
print(f"Our loss: {loss.data}")

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


Our loss: 2.1755153626167143
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535045
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.886450380640099


In [50]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax_t(logits):
  counts = math.e ** logits
  denominator = counts.sum()
  out = counts / denominator
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double()
logits.requires_grad = True
probs = softmax_t(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(f"Torch loss: {loss.item()}")


Torch loss: 2.1755153626167143
